In [1]:
from pyomo.environ import *
from pyomo.dae import *
import numpy as np

m = ConcreteModel()

t_end = 15
m.t = ContinuousSet(bounds=(0,t_end)) 

m.p_nc = Var(m.t,initialize=0, bounds=(-0.1,0.1))

m.Df = Var(m.t, initialize=0, bounds=(-5,5))
m.p_g = Var(m.t, bounds=(-10,10))
m.Df_int = Var(m.t, bounds=(-5,5))
m.e_nc = Var(m.t, initialize=0, bounds=(-10,10))

H = 3.5
K_d = 1.0
Droop = 0.05

m.dDf  = DerivativeVar(m.Df)
m.dp_g = DerivativeVar(m.p_g)
m.dDf_int = DerivativeVar(m.Df_int)
m.de_nc = DerivativeVar(m.e_nc)

def _obj(M):
    return -M.Df_int[t_end] - (1.0-M.e_nc[t_end])
m.obj = Objective(rule=_obj, sense=minimize, doc='Define objective function')

def _dDf(M,i):
    if i == 0:
        return Constraint.Skip
    return M.dDf[i] == 1.0/(2.0*H)*(M.p_g[i] + M.p_nc[i] - 0.2)

m.dxcon = Constraint(m.t, rule=_dDf)

def _dp_g(M,i):
    if i == 0:
        return Constraint.Skip
    return M.dp_g[i] == 1.0/2.0*(-1.0/Droop*M.Df[i] - M.p_g[i])
m.dpgcon = Constraint(m.t, rule=_dp_g)

def _dDf_int(M,i):
    if i == 0:
        return Constraint.Skip
    return M.dDf_int[i] == M.Df[i]
m.dvcon = Constraint(m.t, rule=_dDf_int)

def _de_nc(M,i):
    if i == 0:
        return Constraint.Skip
    return M.de_nc[i] == -M.p_g[i]
m.degcon = Constraint(m.t, rule=_de_nc)

#def _power(M,i):
#    return M.u[i]*M.v[i] < 20.0e6
#m.power = Constraint(m.t, rule=_power)

def _init(M):
    yield M.Df[0] == 0
    yield M.p_g[0] == 0
    yield M.Df_int[0] == 0
    yield M.e_nc[0] == 0
    yield ConstraintList.End
m.init_conditions = ConstraintList(rule=_init)

from pyomo.environ import *
from pyomo.dae import *
from pyomo.opt import SolverFactory
from pyomo.dae.plugins.finitedifference import Finite_Difference_Transformation
from pyomo.dae.plugins.colloc import Collocation_Discretization_Transformation


instance = m

# Discretize model using Backward Finite Difference method
#discretize = Finite_Difference_Transformation()
#disc_instance = discretize.apply(instance,nfe=20,scheme='BACKWARD')

# Discretize model using Orthogonal Collocation
discretize = Collocation_Discretization_Transformation()
disc_instance = discretize.apply(instance,nfe=40,ncp=10,scheme='LAGRANGE-RADAU')

# Will reimplement this method in future release of pyomo.dae
# disc_instance = discretize.reduce_collocation_points(var=instance.u,
# 	ncp=2, diffset=instance.t)

solver='ipopt'
opt=SolverFactory(solver)

results = opt.solve(disc_instance,tee=True)
disc_instance.load(results)

Df = []
p_g = []
p_nc = []
e_nc = []
t=[]

#print(sorted(disc_instance.t))

for i in sorted(disc_instance.t):
    t.append(i)
    Df.append(value(disc_instance.Df[i]))
    p_g.append(value(disc_instance.p_g[i]))
    p_nc.append(value(disc_instance.p_nc[i]))
    e_nc.append(value(disc_instance.e_nc[i]))
t = np.array(t)    
Df = np.array(Df)    
p_g = np.array(p_g)    
p_nc = np.array(p_nc)   
e_nc = np.array(e_nc)   

	Please use either Transformation.apply_to() for in-place transformations
	or Transformation.create_using() for transformations that create a new,
	independent transformed model instance.


AttributeError: 'NoneType' object has no attribute '_transformation_data'

In [47]:
import matplotlib.pyplot as plt




fig, (ax0,ax1,ax2,ax3) = plt.subplots(nrows=4)   # creates a figure with one axe

ax0.plot(t,Df/1.0e3)
ax1.plot(t,p_g, 'g', lw=1)
ax2.plot(t,p_nc, 'b', lw=1)
ax3.plot(t,e_nc, 'b', lw=1)
plt.show()

#fig, (ax0,ax1) = plt.subplots(nrows=2)   # creates a figure with one axe

#ax0.plot(t,u/1.0e3) 
#ax1.plot(t,p/1.0e6) 
#plt.show()
